In [1]:
import tensorflow as tf

# Lista GPUs disponíveis
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Configura a memória da GPU para crescer dinamicamente (opcional)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Configuração de memória ajustada para a GPU.")
    except RuntimeError as e:
        print(f"Erro ao configurar a GPU: {e}")
else:
    print("Nenhuma GPU detectada. Usando CPU.")


Configuração de memória ajustada para a GPU.


In [2]:
from tensorflow.keras import Model # type: ignore
from tensorflow.keras.applications import MobileNetV3Large # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D # type: ignore
from tensorflow.keras.optimizers import Adam; # type: ignore
import numpy as np


In [3]:
train_path = 'C:/Users/carlo/Materias_Optativas/2_Semestre/TETI-Chamy/Teti2024/DataSetMachineLeanig/Atividade/train'
valid_path = 'C:/Users/carlo/Materias_Optativas/2_Semestre/TETI-Chamy/Teti2024/DataSetMachineLeanig/Atividade/validate'

In [4]:
trainGenerator =  ImageDataGenerator(
    rotation_range = 15, width_shift_range = 0.1,
    height_shift_range = 0.1 , brightness_range = (0, 0.2)).flow_from_directory(train_path,target_size = (320,320),batch_size = 32
                                                                               )




validGenerator = ImageDataGenerator(
    rotation_range = 15, width_shift_range = 0.1,
    height_shift_range = 0.1 , brightness_range= (0,0.2)).flow_from_directory(valid_path,target_size = (320,320),batch_size = 32
                                                                             )


Found 1351 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


In [5]:
baseModel = MobileNetV3Large(weights= 'imagenet' , include_top = False,input_shape=(320,320,3))

x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)

predictionLayer = Dense(2,activation= 'softmax')(x)

In [6]:
model = Model(inputs = baseModel.input, outputs = predictionLayer)

In [7]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 320, 320, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 160, 160, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [7]:
for layer in model.layers[:-5]:
    layer.trainable = False

In [9]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 320, 320, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 160, 160, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
#compilando
optimizer = Adam(learning_rate = 0.0001)
model.compile(loss = 'categorical_crossentropy', optimizer =  'Adam', metrics = ['accuracy'])

In [9]:
model.fit(trainGenerator,validation_data=validGenerator,epochs =5)

Epoch 1/5
43/43 [==============================] - 36s 692ms/step - loss: 0.1993 - accuracy: 0.8993 - val_loss: 0.0906 - val_accuracy: 0.9833
Epoch 2/5
43/43 [==============================] - 27s 626ms/step - loss: 0.0861 - accuracy: 0.9608 - val_loss: 0.0696 - val_accuracy: 0.9749
Epoch 3/5
43/43 [==============================] - 28s 652ms/step - loss: 0.0747 - accuracy: 0.9608 - val_loss: 0.0668 - val_accuracy: 0.9582
Epoch 4/5
43/43 [==============================] - 27s 631ms/step - loss: 0.0657 - accuracy: 0.9652 - val_loss: 0.0766 - val_accuracy: 0.9623
Epoch 5/5
43/43 [==============================] - 27s 628ms/step - loss: 0.0645 - accuracy: 0.9608 - val_loss: 0.0573 - val_accuracy: 0.9791


In [10]:
score = model.evaluate(trainGenerator)
print ('Erro no teste: ', round(score[0],5))
print ('Acurácia no teste ', round(score[1],5))

43/43 [==============================] - 23s 529ms/step - loss: 0.0578 - accuracy: 0.9704
Erro no teste:  0.05785
Acurácia no teste  0.97039


In [11]:
model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x000001E913596B90>>

In [12]:

modelSavedPath = 'C:/Users/carlo/OneDrive/Documentos/Python/PythonJupyterPy310/MachineLeaning/Atividade/VeiculoV3.h5Large/'
model.save(modelSavedPath)

INFO:tensorflow:Assets written to: C:/Users/carlo/OneDrive/Documentos/Python/PythonJupyterPy310/MachineLeaning/Atividade/VeiculoV3.h5Large/assets


INFO:tensorflow:Assets written to: C:/Users/carlo/OneDrive/Documentos/Python/PythonJupyterPy310/MachineLeaning/Atividade/VeiculoV3.h5Large/assets
